In [ ]:
# Uncomment to install chat2func if not installed
# !pip install chat2func[develop]

# chat2func

What does it do?
- Automatically generate JSON schemas of python functions
- Integrate into ChatGPT plugins
- Automate function calling in python
- Convert unstructured data to structured data

In [ ]:
from pprint import pprint as print
from chat2func import json_schema

@json_schema
def addition(x: float, y: float) -> float:
    """adds two numbers

    Args:
        x (float): first number
        y (float): second number
    """
    return x + y

print(addition.__schema__)

# Make it a Plugin

In [ ]:
from chat2func.server import FunctionServer

server = FunctionServer({"addition": addition})
server.run(debug=True, use_reloader=False) # use_reloader=False for Jupyter

### Go to [ChatGPT](https://chat.openai.com/?model=gpt-4-plugins)

- Make sure you have the plugin version of ChatGPT selected.
- Click on the Plugin Store.
- Click `Develop your own plugin` in the bottom right
- Put in `localhost:3333` (or another port if you changed it from the default)
- Ask ChatGPT questions about your plugin!

![Plugin Demo](./data/plugin-demo.png)

# Export (and Deploy!)

It's just a flask app. You could use a simple google cloud function to deploy it.

In [ ]:
# Export the main plugin files.
server.export("./my-plugin")

# Using the Function Calling API

Here we demonstrate the power of the function calling API by converting unstructured data to structured data, in the form of a python dataclass.

_Note: the `FunctionCallingAPI` is still experimental._

In [ ]:
# Set your API key
# import openai
# openai.api_key = "..."

In [ ]:
# Get the imports!
from dataclasses import dataclass
from typing import Optional
from chat2func import function_calls
from chat2func.api import FunctionCallingAPI, Role

@json_schema
@dataclass
class Character:
    name: str
    age: Optional[int] = None
    house: Optional[str] = None

In [ ]:
# Setup our function calling API
system_prompt = "You are a literary expert on the Dune book series."
functions = {"Character": Character}
api = FunctionCallingAPI(functions, system_prompt)
api.chat.add_message(Role.USER.value, "List the heads of houses in the Dune series. Give best estimates of age's where appropriate.")

In [ ]:
for _ in range(5):
    # Force the API to call a function
    message = api.reply(force_function="Character")
    api.chat.messages.append(message)

    # Call the function (and validate the inputs!)
    fn_name = message.function_call["name"]
    args = message.function_call["arguments"]
    result = function_calls(fn_name, functions, args, return_json=False)

    # Add the result to the chat
    api.chat.add_message(Role.FUNCTION.value, str(result), name="character")
    print(result)